In [19]:
import sys
from pathlib import Path
import json
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm

# Add project root to path
sys.path.insert(0, '/home/research/workspace/VisualCoT-NLE/src')

# Import CLIP manager
from vctp.utils.clip_manager import get_clip_model

# Define paths (giống như trong script)
BASE_DIR = "/home/research/workspace/VisualCoT-NLE"
VIVQA_PATH = "/home/research/workspace/data/raw/vivqa-x/annotations"
VIVQA_IMAGES = "/home/research/workspace/data/raw/coco/images"
OUTPUT_DIR = f"{BASE_DIR}/data/processed"

# Test với một split (ví dụ: test)
SPLIT_ANNOTATION = "train"
SPLIT_IMAGE = "train"
# 1. Test make_line2sample
print(f"Testing line2sample for {SPLIT_ANNOTATION}...")
input_file = f"{VIVQA_PATH}/{SPLIT_ANNOTATION}.json"
inputs = json.load(open(input_file))

output_dict = {}
for idx, q in enumerate(inputs):
    val = "<->".join([str(q['image_id']), str(q['question_id'])])
    output_dict[str(idx)] = val

print(f"Created {len(output_dict)} line2sample mappings")
print("Sample:", list(output_dict.items()))

# 2. Test CLIP features (với subset nhỏ để test nhanh)
print(f"\nTesting CLIP features for {SPLIT_ANNOTATION}...")
dataset = json.load(open(input_file))


# Chỉ test với 5 samples đầu tiên
dataset_sample = dataset[:5]
print(dataset_sample)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model, processor = get_clip_model(
    model_type="full", device=device, use_safetensors=True
)

text_embeds_list = []
image_embeds_list = []

for q in tqdm(dataset_sample):
    imageID = q["image_id"]
    imageName = q["image_name"]
    file_name = VIVQA_IMAGES + "/" + f"{SPLIT_IMAGE}2014" +"/" + imageName
    image = Image.open(file_name)
    inputs = processor(
        text=[q["question"]], images=image, return_tensors="pt", padding=True
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    text_embeds_list.append(outputs["text_embeds"].detach().cpu())
    image_embeds_list.append(outputs["image_embeds"].detach().cpu())

text_embeds = torch.cat(text_embeds_list, dim=0)
image_embeds = torch.cat(image_embeds_list, dim=0)

print(f"\nText embeddings shape: {text_embeds.shape}")
print(f"Image embeddings shape: {image_embeds.shape}")

Testing line2sample for train...
Created 29459 line2sample mappings
Sample: [('0', '262146<->262146001'), ('1', '393223<->393223000'), ('2', '393230<->393230000'), ('3', '393230<->393230003'), ('4', '137045<->137045002'), ('5', '458763<->458763004'), ('6', '567990<->567990002'), ('7', '393286<->393286001'), ('8', '71<->71000'), ('9', '71<->71001'), ('10', '524375<->524375001'), ('11', '524377<->524377007'), ('12', '92<->92001'), ('13', '546151<->546151001'), ('14', '21826<->21826002'), ('15', '110<->110002'), ('16', '127<->127001'), ('17', '142<->142000'), ('18', '131215<->131215001'), ('19', '144<->144002'), ('20', '149<->149001'), ('21', '262308<->262308002'), ('22', '131245<->131245002'), ('23', '393394<->393394001'), ('24', '393394<->393394003'), ('25', '393396<->393396000'), ('26', '393396<->393396001'), ('27', '194<->194002'), ('28', '393418<->393418001'), ('29', '393432<->393432000'), ('30', '131300<->131300001'), ('31', '131312<->131312001'), ('32', '524551<->524551002'), ('33'

100%|██████████| 5/5 [00:00<00:00, 48.96it/s]


Text embeddings shape: torch.Size([5, 512])
Image embeddings shape: torch.Size([5, 512])


In [23]:
# test_similarity_retrieval.py
import json
import sys
from pathlib import Path

# Add project root to path
sys.path.insert(0, '/home/research/workspace/VisualCoT-NLE/src')

# Import class của bạn
from vctp.think.context.similarity_retriever import SimilarityRetriever

# ====================
# CONFIG
# ====================
BASE_DIR = "/home/research/workspace/VisualCoT-NLE"
SIMILARITY_PATH = f"{BASE_DIR}/data/processed"
VIVQA_PATH = "/home/research/workspace/data/raw/vivqa-x/annotations"

DATASET_NAME = "vivqax"
SPLIT = "val"  # hoặc "test"

# ====================
# 1. KHỞI TẠO RETRIEVER
# ====================
print("=" * 60)
print("TESTING SIMILARITY RETRIEVAL")
print("=" * 60)

retriever = SimilarityRetriever(
    similarity_path=SIMILARITY_PATH,
    dataset_name=DATASET_NAME,
    split=SPLIT
)

# Load features
retriever.load_features(metric="imagequestion")

# ====================
# 2. LOAD VAL DATASET ĐỂ LẤY QUERY EXAMPLES
# ====================
val_file = f"{VIVQA_PATH}/{SPLIT}.json"
val_data = json.load(open(val_file))

# Tạo dict để tra cứu nhanh
val_dict = {}
for item in val_data:
    key = f"{item['image_id']}<->{item['question_id']}"
    val_dict[key] = item

# ====================
# 3. TEST VỚI MỘT VÀI QUERIES
# ====================
print("\n" + "=" * 60)
print("TESTING RETRIEVAL RESULTS")
print("=" * 60)

# Lấy 3 queries đầu tiên để test
test_queries = list(val_dict.keys())[:3]

# Load train data một lần
train_file = f"{VIVQA_PATH}/train.json"
train_data = json.load(open(train_file))
train_dict = {}
for item in train_data:
    key = f"{item['image_id']}<->{item['question_id']}"
    train_dict[key] = item

for query_key in test_queries:
    print(f"\n{'=' * 60}")
    print(f"QUERY: {query_key}")
    query_item = val_dict[query_key]
    print(f"  Image: {query_item['image_name']}")
    print(f"  Question: {query_item['question']}")
    
    # Get similar examples (top 5)
    similar_keys = retriever.get_similar_examples(
        query_key=query_key,
        metric="imagequestion",
        n_shot=5
    )
    print("Similar keys:", similar_keys)
    print(f"\n  TOP 5 SIMILAR EXAMPLES:")
    print(f"  {'-' * 56}")
    for i, similar_key in enumerate(similar_keys, 1):
        if similar_key in train_dict:
            similar_item = train_dict[similar_key]
            print(f"\n  [{i}] {similar_key}")
            print(f"      Image: {similar_item['image_name']}")
            print(f"      Question: {similar_item['question'][:80]}...")
            if 'answer' in similar_item:
                print(f"      Answer: {similar_item['answer']}")
        else:
            print(f"\n  [{i}] {similar_key} (not found in train data)")

# ====================
# 4. TEST SIMILARITY SCORES
# ====================
print(f"\n\n{'=' * 60}")
print("TESTING SIMILARITY SCORES")
print("=" * 60)

query_key = test_queries[0]
print(f"\nQuery: {query_key}")

# Get all scores
all_scores = retriever.get_similar_with_scores(
    query_key=query_key,
    metric="imagequestion"
)

# Sort và lấy top 10
sorted_scores = sorted(all_scores.items(), key=lambda x: x[1], reverse=True)[:10]

print(f"\nTOP 10 WITH SCORES:")
print(f"{'-' * 60}")
for key, score in sorted_scores:
    print(f"  {key}: {score:.4f}")

# Thống kê
import numpy as np
scores_array = np.array(list(all_scores.values()))
print(f"\nSCORE STATISTICS:")
print(f"  Mean: {scores_array.mean():.4f}")
print(f"  Std: {scores_array.std():.4f}")
print(f"  Min: {scores_array.min():.4f}")
print(f"  Max: {scores_array.max():.4f}")

print("\n" + "=" * 60)
print("TEST COMPLETED!")
print("=" * 60)

TESTING SIMILARITY RETRIEVAL
Loading vivqax features from /home/research/workspace/VisualCoT-NLE/data/processed...
  Loading val index: /home/research/workspace/VisualCoT-NLE/data/processed/vivqax_qa_line2sample_idx_val.json
  Loading train question features: /home/research/workspace/VisualCoT-NLE/data/processed/coco_clip_vitb16_train_vivqax_question.npy
  Loading val question features: /home/research/workspace/VisualCoT-NLE/data/processed/coco_clip_vitb16_val_vivqax_question.npy
  Loading train index: /home/research/workspace/VisualCoT-NLE/data/processed/vivqax_qa_line2sample_idx_train.json
  Loading train image features: /home/research/workspace/VisualCoT-NLE/data/processed/coco_clip_vitb16_train_vivqax_convertedidx_image.npy
  Loading val image features: /home/research/workspace/VisualCoT-NLE/data/processed/coco_clip_vitb16_val_vivqax_convertedidx_image.npy
✓ Loaded features successfully!
  Train samples: 29459
  Val samples: 1459

TESTING RETRIEVAL RESULTS

QUERY: 393271<->39327100